# Probability
## Final project

In the final project you will develop your own classification algorithm based on probabilistic models.

In the `data` folder you can find a collection of texts in English, Italian, Spanish, German, French, Polish and Portuguese languages obtained from random Wikipedia articles, see e.g. `Spanish.txt`. (See corresponding `.source.txt` files and links therein for lists of authors.) 

In [1]:
!ls data

English.sources.txt  German.txt		  Portuguese.sources.txt
English.txt	     Italian.sources.txt  Portuguese.txt
French.sources.txt   Italian.txt	  Spanish.sources.txt
French.txt	     Polish.sources.txt   Spanish.txt
German.sources.txt   Polish.txt


In [2]:
with open("data/Spanish.txt") as f:
    print(f.read(500))

tomas zapata sierra medellin colombia  de mayo de  es productor de cine y de teatrocita requerida
sad eyed lady of the lowlands en espanol senorita de ojos tristes de las tierras bajas es una cancion compuesta por el cantante estadounidense bob dylan fue incluida en el album blonde on blonde editado el  de mayo de 
la revista mojo la coloco en el puesto  de su lista de las  mejores canciones de bob dylan
calyptocephalella canqueli es una especie extinta de anfibio anuro perteneciente al genero c


In [3]:
with open("data/German.txt") as f:
    print(f.read(500))

riedhofe ist der name von ortsteilen in deutschland

in badenwurttemberg
riedhofe langenau ortsteil der stadt langenau im albdonaukreis
riedhofe frickingen ortsteil der gemeinde frickingen im bodenseekreis
riedhofe riegel am kaiserstuhl ortsteil der gemeinde riegel am kaiserstuhl im landkreis emmendingen
riedhofe kongen ortsteil der gemeinde kongen im landkreis esslingen
riedhofe leingarten ortsteil der gemeinde leingarten im landkreis heilbronn
riedhofe bad wurzach ortsteil der stadt bad wurzac


This is our training data. These texts are preprocessed: only standard Latin characters kept, diacritics removed, punctuations removed, all letters converted to lowercase. We will use similar preprocessing for new texts that we have to classify. Here are some useful functions to do it.

### Preprocessing functions

In [4]:
import re
import unicodedata

### FROM: https://stackoverflow.com/a/518232/3025981
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')
### END FROM

def clean_text(s):
    return re.sub("[^a-z \n]", "", strip_accents(s))

### Learning: obtaining character frequencies

First of all, we have to find character relative frequencies in texts of each language. We will consider them as probability for character to appear in our multinomial model. Write function `get_freqs(text, relative)` that takes string `text` as input and returns dictionary which keys are all distinct characters occurred in `text` and values are frequencies (relative if `relative` is `True` and absolute otherwise). A similar function was discussed in Python videos previously. Note that Python treat `str` objects as a sequence of characters (e.g. if you try to iterate it).

In [6]:
from collections import Counter

def get_freqs(text, relative=False):
    counter = Counter(text)
    
    if relative:
        total_count = sum(counter.values())
        relative_counter = {}
        for key in counter:
            relative_counter[key] = counter[key] / total_count
        return relative_counter
    else:
        return counter

In [7]:
assert get_freqs('Hello, World!') == {'H': 1, 'e': 1, 'l': 3, 'o': 2, ',': 1,
                                      ' ': 1, 'W': 1, 'r': 1, 'd': 1, '!': 1}

Now use function `get_freqs` to create a dictionary `lang_to_prob` which keys are names of languages (i.e. `'English'`, `'Italian'`, `'Spanish'`, `'German'`, `'French'`, `'Polish'`, `'Portuguese'`) and values are dictionaries of relative frequencies, obtained by processing of corresponding `.txt` files.

In [8]:
# your code here
lang_to_probs = {}

for lang in ('English', 'French', 'German', 'Italian', 'Polish', 'Portuguese', 'Spanish'):
    with open("data/{}.txt".format(lang)) as f:
        lang_to_probs[lang] = get_freqs(f.read(), relative=True)

In [9]:
assert abs(lang_to_probs['Polish']['a'] - 0.08504245058355897) < 0.00001
assert abs(lang_to_probs['English']['x'] - 0.001387857977519179) < 1e-5
assert len(set(lang_to_probs['Portuguese'])) == 28

### Likelihoods
Now let us start implementing the actual classifier. We begin with a multinomial likelihood function. Implement function `multinomial_likelihood(probs, freqs)` that takes two arguments: `probs` are dictionary of probabilities of each character (in some language) and `freqs` is dictionary of absolute frequencies of each character (in some text we want to classify). This function has to return probability to obtain these absolute frequencies from multinomial distribution with given probabilities $P((X_1 = f_1) \cap (X_2 = f_2) \cap \ldots \cap (X_k = f_k))$ provided that $(X_1, \ldots, X_k)$ is a system of multinomially distributed values with probabilities $(p_1, \ldots, p_k)$. (You need the `factorial` function that can be imported from `math`.)


In [10]:
import math
# your code here
def multinomial_likelihood(probs, freqs):
    n = sum(freqs.values())
    p1 = math.factorial(n)
    p2 = 1
    p3 = 1
    
    for k in probs.keys() & freqs.keys():
        p2 *= math.factorial(freqs[k])
        p3 *= probs[k] ** freqs[k]

    return p1 * p3 / p2

Let's find likelihood of data with frequencies `{'a': 2, 'b': 1, 'c': 2}` and probabilities `{'a': 0.2, 'b': 0.5, 'c': 0.3}`:

In [11]:
multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3}, freqs={'a': 2, 'b': 1, 'c': 2})

0.05400000000000001

In [12]:
assert abs(multinomial_likelihood(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 2, 'b': 1, 'c': 2}) - 0.054) < 0.000001
assert abs(multinomial_likelihood(probs={'a': 0.2, 'b': 0.1, 'c': 0.3, 'd': 0.4},
                    freqs={'a': 2, 'b': 1, 'c': 2}) - 0.0108) < 0.000001

Note that the coefficient with factorials depends only on `freqs` (i.e. text that we analyse) and does not depend on `probs`. It means that for all possible languages this coefficient will be the same. As we are going to consider fixed text and compare likelihoods for different languages, we see that we do not need this coefficient in most cases. Let us implement the function `multinomial_likelihood_without_coeff` that returns the same probability as `multinomial_likelihood` but without the coefficient.

In [14]:
# your code here
def multinomial_likelihood_without_coeff(probs, freqs):
    p = 1
    
    for k in probs.keys() & freqs.keys():
        p *= probs[k] ** freqs[k]

    return p

The corresponding probability becomes smaller:

In [15]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 2, 'b': 1, 'c': 2})

0.0018000000000000004

In [16]:
assert multinomial_likelihood_without_coeff(
    probs={'a': 0.3, 'b': 0.4, 'c': 0.3},
    freqs={'a': 2, 'b': 1, 'c': 2}) == 0.00324
assert abs(multinomial_likelihood_without_coeff(
    probs={'a': 0.3, 'b': 0.4, 'c': 0.3},
    freqs={'a': 2, 'b': 1, 'c': 5}) - 8.747999999999e-05) < 1e-10

Actually, likelihoods become extremely small very quickly when we increase absolute frequencies in data. It is not surprising: the probability to get the text that coincides with our actual text from a random experiment we discussed is extremely small.

In [17]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 2, 'c': 2})

0.00018000000000000004

In [18]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                    freqs={'a': 3, 'b': 20, 'c': 2})

6.866455078125001e-10

Due to limited precision of computer arithmetic, for frequencies large enough we will get exactly zero likelihood.

In [19]:
multinomial_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                                     freqs={'a': 543, 'b': 512, 'c': 2})

0.0

Thus we usually cannot use likelihoods like this directly. Common way to deal with such a tiny numbers is to use _logarithms_ instead of likelihood themself. Indeed, logarithm is a monotonically increasing function. If we compare logarithms, it is equivalent to compare their arguments.

### Log likelihood
Implement function `log_likelihood_without_coeff(probs, freqs)` that calculates logarithm of likelihood (without factorial coefficient). Note that you cannot simply put `multinomial_likelihood_without_coeff` into `log`: if the likelihood would be extremely small (equal to zero from the computer's point of view), `log` of it will be negative infinity. Thus we have to algebraically transform the expression for log likelihood first. Let us use properties of logarithm to do it: $\log (ab) = \log a + \log b$, $\log (a^b)=b\log a$.

In [20]:
# your code here
def log_likelihood_without_coeff(probs, freqs):
    p = 0
    for k in probs.keys() & freqs.keys():
            p += freqs[k] * math.log(probs[k])
    return p

Likelihoods are probabilities, so they are less than 1 and their logarithms are negative. The larger absolute value of log-likelihood, the less is likelihood. 

In [21]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 2, 'b': 1, 'c': 2})

-6.319968614080018

Now we can deal with the inputs that lead to exact zero value previously.

In [22]:
log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2})

-1231.2240885070603

In [23]:
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                              freqs={'a': 2, 'b': 1, 'c': 2}) + 6.319968614080018) < 0.00001
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512, 'c': 2}) + 1231.2240885070605) < 0.0001
assert abs(log_likelihood_without_coeff(probs={'a': 0.2, 'b': 0.5, 'c': 0.3},
                             freqs={'a': 543, 'b': 512}) + 1228.8161428984085) < 0.0001

### Theoretical question: cross-entropy and Jensen's inequality

The function that you obtained is also known as *cross entropy* and is extremely popular in machine learning, namely, in classification problems. It allows you to measure how good probability distribution $(p_1, \ldots, p_k)$ fits to actual absolute frequencies obtained from the data $(f_1, \ldots, f_k)$.

Assume that frequencies $(f_1, \ldots, f_k)$ are fixed. What is the best distribution $(p_1, \ldots, p_k)$ from likelihood's perspective?

Intuitively, it seems that we have to put relative frequencies
$$r_i = \frac{f_i}{\sum_{j=1}^k f_j}$$ 
as $p_i$ to get best fit. In fact, it is true. To prove it, let us use Jensen's inequality for logarithms. It is stated as follows:

For any values $\alpha_1, \ldots, \alpha_k$, such that $\sum_{j=1}^k \alpha_j = 1$ and $\alpha_j \ge 0$ for all $j=1, \ldots, k$, and any positive values $x_1, \ldots, x_k$, the following inequality holds:

$$\log \sum_{j=1}^k \alpha_j x_j \ge \sum_{j=1}^k \alpha_j\log(x_j).$$

Use this inequality to prove that
$$\sum_{j=1}^k r_j \log p_j - \sum_{j=1}^k r_j \log r_j \le 0,$$
then prove that to obtain maximum log-likelihood (and therefore maximum likelihood) for fixed $(f_1, \ldots, f_k)$ we have to put $p_i=r_i$, $i=1,\ldots, k$.

**Hint:** use properties of logarithm to transform the left-hand part of the last inequality to the right-hand part of the previous inequality.

(Submit your proof as a staff graded assignment.)

### Maximum likelihood classifier

Now we will use likelihood to choose the best language for some text we want to classify. Write function `mle_best(text, lang_to_probs)` that takes some `text` and dictionary `lang_to_probs` that we created previously and returns the name of the language such that the likelihood of our data for this language is maximal. Note that you have to preprocess  `text` using the function `clean_text` before finding frequencies.

In [50]:
# your code here
def mle_best(text, lang_to_probs, return_all=False):
    freqs = get_freqs(clean_text(text), True)
    
    p = {}
    
    for lang, probs in lang_to_probs.items():
        p[lang] = log_likelihood_without_coeff(probs=probs, freqs=freqs)
    if return_all: 
        return p
    
    return max(p, key=lambda key: p[key])

Let's test it!

In [25]:
# Source: https://en.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler, provisional designation 1929 SA, is a stony asteroid 
and eccentric Mars-crosser from the asteroid belt, approximately 
4 kilometers in diameter"""
mle_best(text, lang_to_probs)

'English'

In [26]:
# Source: https://pl.wikipedia.org/wiki/(1134)_Kepler
text = """"(1134) Kepler – planetoida z grupy przecinających 
orbitę Marsa okrążająca Słońce w ciągu 4 lat i 145 dni 
w średniej odległości 2,68 au.
"""
mle_best(text, lang_to_probs)

'Polish'

In [27]:
# Source: https://it.wikipedia.org/wiki/1134_Kepler
text = """1134 Kepler è un asteroide areosecante. Scoperto nel 1929, 
presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 
UA e da un'eccentricità di 0,4651458, inclinata di 15,17381° rispetto
"""
mle_best(text, lang_to_probs)

'Italian'

In [28]:
lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]
assert mle_best(lines[0], lang_to_probs) == 'Portuguese'
assert mle_best(lines[1], lang_to_probs) == 'Portuguese'
assert mle_best(lines[2], lang_to_probs) == 'French'
assert mle_best(lines[3], lang_to_probs) == 'Italian'
assert mle_best(lines[4], lang_to_probs) == 'Polish'
assert mle_best(lines[5], lang_to_probs) == 'English'
assert mle_best("Aaaa", lang_to_probs) == "Portuguese"

### Look, it works!
Fantastic! Just a couple of equations, and we created fully automatic language detection algorithm!

Now let us make it even better.

### Let's go Bayes
Assume that we selected random article from all Wikipedia articles in the languages that we consider. There are different numbers of articles in different languages, so it is more likely to obtain an article e.g. in English than in Polish (at least, at this moment). It means that we need stronger evidence for Polish to accept it compared with English. To take into account this information, we will use Bayes' rule as discussed in the videos.

Recall that in the Bayesian approach we consider _prior_ probabilities of languages, then use Bayes' rule to find their posterior probabilities and select the language with the largest posterior probability. We begin with finding prior probabilities. Create a dictionary `lang_to_prior` whose keys are language names and values are prior probabilities. Assume that priors are proportional to the number of articles in each language (in thousands). Let us use these numbers (in thousands): English: 6090, Italian: 1611, Spanish: 1602, German: 2439, French: 2222, Polish: 1412, Portuguese: 1034. Remember that all prior probabilities should sum up to 1!

In [33]:
# your code here
lang_articles = {
    'English': 6090,
    'Italian': 1611,
    'Spanish': 1602,
    'German': 2439,
    'French': 2222,
    'Polish': 1412,
    'Portuguese': 1034
}

articles_ttl = sum(lang_articles.values())

lang_to_prior = {key: lang_articles[key]/articles_ttl for key in lang_articles.keys()}

print(lang_to_prior)

{'English': 0.3711151736745887, 'Italian': 0.09817184643510055, 'Spanish': 0.09762340036563072, 'German': 0.1486288848263254, 'French': 0.13540524070688603, 'Polish': 0.08604509445460086, 'Portuguese': 0.06301035953686776}


In [34]:
assert abs(lang_to_prior['French'] - 0.13540524070688603) < 0.000001

Now implement function `bayesian_best(text, lang_to_probs, lang_to_prior)` that takes some text `text`, dictionary `lang_to_probs` created before and `lang_to_prior` with prior probabilities. This function has to return the language name with the largest posterior probability. Note that as we only compare posterior probabilities, we can ignore the denominator in Bayes' rule: it is the same for all languages. Note also that we need to work with logarithms of posterior instead of posteriors themself to avoid extremely small numbers. Use properties of logarithm to do it efficiently.

In [55]:
# your code here
def bayesian_best(text, lang_to_probs, lang_to_prior):
    freqs = get_freqs(clean_text(text), True)
    likelihood = mle_best(text, lang_to_probs, True)
    res = {}

    for key in likelihood.keys():
        print('key', key, likelihood[key], math.log(lang_to_prior[key]), likelihood[key] + math.log(lang_to_prior[key]))
        res[key] = likelihood[key] + math.log(lang_to_prior[key])
    

    return max(res, key=lambda key: res[key])

lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]
answers = ['English', 'English', 'French', 'Italian', 'Polish', 'English']

for line, answer in zip(lines, answers):
    print(bayesian_best(line, lang_to_probs, lang_to_prior))

key English -2.556093999491036 -0.9912428233801939 -3.54733682287123
key French -2.6419785979002963 -1.9994832138845613 -4.641461811784858
key German -2.8304768154846425 -1.9063027858682162 -4.7367796013528585
key Italian -2.342416968769556 -2.321035800907162 -4.663452769676718
key Polish -2.464604728529693 -2.4528837660309493 -4.917488494560642
key Portuguese -2.2157575201191277 -2.764456129015762 -4.98021364913489
key Spanish -2.30698254150174 -2.326638056455832 -4.633620597957572
English
key English -2.556093999491036 -0.9912428233801939 -3.54733682287123
key French -2.6419785979002963 -1.9994832138845613 -4.641461811784858
key German -2.8304768154846425 -1.9063027858682162 -4.7367796013528585
key Italian -2.342416968769556 -2.321035800907162 -4.663452769676718
key Polish -2.464604728529693 -2.4528837660309493 -4.917488494560642
key Portuguese -2.2157575201191277 -2.764456129015762 -4.98021364913489
key Spanish -2.30698254150174 -2.326638056455832 -4.633620597957572
English
key Engl

For example, MLE algorithm believes that word `"The"` belongs go German language.

In [29]:
mle_best("The", lang_to_probs)

'German'

However, if we take into account that English is more popular in Wikipedia, the results changes:

In [57]:
bayesian_best("The", lang_to_probs, lang_to_prior)

key English -2.8434741872582263 -0.9912428233801939 -3.8347170106384203
key French -3.3039505374577938 -1.9994832138845613 -5.3034337513423555
key German -2.730045843356664 -1.9063027858682162 -4.63634862922488
key Italian -3.5816633147693637 -2.321035800907162 -5.902699115676526
key Polish -3.52414439992688 -2.4528837660309493 -5.9770281659578295
key Portuguese -3.5079318432481092 -2.764456129015762 -6.272387972263871
key Spanish -3.6002692362659103 -2.326638056455832 -5.926907292721742


'English'

In [58]:
lines = ['Aaa', 'Aa', 'Kepler è un asteroide areosecante. Scoperto nel',
        "presenta un'orbita caratterizzata da un semiasse maggiore pari a 2,6829098 "
         "UA e da un'eccentricità di 0,4651458, inclinata",
         "Kepler – planetoida z grupy przecinających orbitę Marsa okrążająca Słońce w ciągu 4 lat i ",
         "Kepler, provisional designation 1929 SA, is a stony asteroid "
         "and eccentric Mars-crosser from the"]
answers = ['English', 'English', 'French', 'Italian', 'Polish', 'English']
for line, answer in zip(lines, answers):
    assert bayesian_best(line, lang_to_probs, lang_to_prior) == answer

key English -2.556093999491036 -0.9912428233801939 -3.54733682287123
key French -2.6419785979002963 -1.9994832138845613 -4.641461811784858
key German -2.8304768154846425 -1.9063027858682162 -4.7367796013528585
key Italian -2.342416968769556 -2.321035800907162 -4.663452769676718
key Polish -2.464604728529693 -2.4528837660309493 -4.917488494560642
key Portuguese -2.2157575201191277 -2.764456129015762 -4.98021364913489
key Spanish -2.30698254150174 -2.326638056455832 -4.633620597957572
key English -2.556093999491036 -0.9912428233801939 -3.54733682287123
key French -2.6419785979002963 -1.9994832138845613 -4.641461811784858
key German -2.8304768154846425 -1.9063027858682162 -4.7367796013528585
key Italian -2.342416968769556 -2.321035800907162 -4.663452769676718
key Polish -2.464604728529693 -2.4528837660309493 -4.917488494560642
key Portuguese -2.2157575201191277 -2.764456129015762 -4.98021364913489
key Spanish -2.30698254150174 -2.326638056455832 -4.633620597957572
key English -2.703603783

AssertionError: 

### Measuring uncertainty
Finally, let us get posterior probability of how certain our Bayesian algorithm is in its classification. Implement function `bayesian_posterior(text, lang_to_probs, lang_to_prior, test_lang)` that takes some `text`, `lang_to_probs`, `lang_to_prior` and a particular language `test_lang` we are interested in and returns posterior probability for this language provided this text. Use formula for the posterior from the videos.

Note that in this case you have to work with actual probabilities (likelihoods), not their logarithms.

In [ ]:
# your code here


In [ ]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "German")

In [ ]:
bayesian_posterior("The", lang_to_probs, lang_to_prior, "English")

In [ ]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English")

In [ ]:
bayesian_posterior("Das", lang_to_probs, lang_to_prior, "German")

In [ ]:
assert abs(bayesian_posterior("The", lang_to_probs, lang_to_prior, "German") - 0.26863814640) < 0.00001
assert abs(bayesian_posterior("The", lang_to_probs, lang_to_prior, "English") - 0.534626604) < 0.00001
assert abs(bayesian_posterior("Das", lang_to_probs, lang_to_prior, "English") - 0.365967931517) < 0.00001

We see that our algorithm believes that `"Das"` belongs to English. This is due to the small amount of data (only three letters!) and high prior for English. However, it is not very certain: the posterior is only 0.37. On the other hand, `"The"` belongs to `"English"` with much larger posterior probability.

## Conclusions
Today we used our knowledge of probability to construct our own classifier algorithm. Actually, it is a version of a well-known naive Bayesian classifier. Of course, this classifier is far from perfect: for example, it completely ignores words and deals only with characters and their frequencies. Nevertheless, it works rather well despite being so simple. Also it shows several important concepts: likelihood, maximum likelihood estimation, Bayesian estimations and so on.

Now you are ready for more sophisticated topics.

### P.S. Efficiency considerations
During this project we used dictionaries to represent frequency tables and pure Python constructions like loops to process them. However, it is much more efficient to use `numpy` arrays and vectorized functions. You can try to redo this project with `numpy` if you know how to use it. If you don't know yet, that's not a problem: you will learn it soon.